## REF 
- https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/2_BasicModels/random_forest.ipynb

In [1]:
# OP 
import numpy as np
import pandas as pd 
from sklearn import datasets
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# DL 
import tensorflow as tf
from tensorflow.python.ops import resources
from tensorflow.contrib.tensor_forest.python import tensor_forest


/Users/yennanliu/anaconda3/envs/ds_dash/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# GENERATE TOY DATA 
# n_classes = 4 
# n_samples = 10000
# n_features = 20 
X, y = datasets.make_classification(n_samples=10000, n_features=20,n_informative=2,
                                    n_clusters_per_class=1,n_redundant=5,n_classes=4)

In [3]:
# TRAIN-TEST SPLIT 

num_data = len(X)
train_split = 0.9
num_train = int(train_split * num_data)
num_test = num_data - num_train
x_train = X[0:num_train]
x_test = X[num_train:]
y_train = y[0:num_train]
y_test = y[num_train:]


print("Size of:")
print("- Training-set:\t\t{}".format(x_train.shape))
#print("- Validation-set:\t{}".format(data.num_val))
print("- Test-set:\t\t{}".format(x_test.shape))


Size of:
- Training-set:		(9000, 20)
- Test-set:		(1000, 20)


In [4]:
# Parameters
num_steps = 100 # Total steps to train
batch_size = 100 # The number of samples per batch
num_classes = 10 # The 10 digits
num_features = 20 # Each image is 28x28 pixels
num_trees = 10
max_nodes = 1000

# Input and Target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.int32, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

In [5]:
# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_outputs': 1, 'bagged_features': None, 'bagged_num_features': 20, 'base_random_seed': 0, 'num_output_columns': 11, 'max_fertile_nodes': 0, 'num_features': 20, 'num_classes': 10, 'early_finish_check_every_samples': 0, 'split_pruning_name': 'none', 'param_file': None, 'checkpoint_stats': False, 'collate_examples': False, 'inference_tree_paths': False, 'pruning_type': 0, 'leaf_model_type': 0, 'valid_leaf_threshold': 1, 'use_running_stats_method': False, 'split_finish_name': 'basic', 'stats_model_type': 0, 'split_type': 0, 'finish_type': 0, 'dominate_fraction': 0.99, 'max_nodes': 1000, 'feature_bagging_fraction': 1.0, 'num_trees': 10, 'num_splits_to_consider': 10, 'prune_every_samples': 0, 'model_name': 'all_dense', 'split_after_samples': 250, 'dominate_method': 'bootstrap', 'initialize_average_splits': False, 'bagging_fraction': 1.0, 'split_name': 'less_or_equal', 'regression': False}


In [6]:
### HELP FUNC 

def get_next_batch(X_train,Y_train):
    idxs = np.random.permutation(X_train.shape[0]) #shuffled ordering
    X_random_batch  = X_train[idxs]
    Y_random_batch = Y_train[idxs]
    return X_random_batch,  Y_random_batch


def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

In [7]:
# Start TensorFlow session
sess = tf.train.MonitoredSession()

# Run the initializer
sess.run(init_vars)

# Training
for i in range(1, num_steps + 1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    ####################################################################################
    #                                                                                  #
    # MODIFY THE CODE HERE (MNIST -> USER DEFINED DATASET)                             #
    ####################################################################################
    
    batch_x, batch_y = get_next_batch(x_train,y_train)
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Model
test_x, test_y = x_test, y_test
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))


Step 1, Loss: -1.000000, Acc: 0.718000
Step 50, Loss: -1001.000000, Acc: 0.914000
Step 100, Loss: -1001.000000, Acc: 0.914000
Test Accuracy: 0.897


In [8]:
# end of analysis 